In [1]:
import joblib
from flask import Flask, request, json, jsonify, render_template
from werkzeug.exceptions import HTTPException

In [16]:
print('Loading model...')
MODEL_PATH = 'models/classifier.joblib'
print('Done!')

Loading model...
Done!


In [17]:
print('Loading app...')
app = Flask(__name__)
print('Done!')

Loading app...
Done!


In [18]:
@app.errorhandler(HTTPException)
def handle_exception(e):
    """Return JSON instead of HTML for HTTP errors (which is the basic error
    response with Flask).
    """
    # Start with the correct headers and status code from the error
    response = e.get_response()
    # Replace the body with JSON
    response.data = json.dumps({
        "code": e.code,
        "name": e.name,
        "description": e.description,
    })
    response.content_type = "application/json"
    return response

In [19]:
class MissingKeyError(HTTPException):
    # We can define our own error for the missing key
    code = 422
    name = "Missing key error"
    description = "JSON content missing key 'input'."

In [20]:
class MissingJSON(HTTPException):
    # We can define our own error for missing JSON
    code = 400
    name = "Missing JSON"
    description = "Missing JSON."

In [21]:
input_simple = {"input": [[7.0, 0.27, 0.36, 20.7, 0.045, 45.0, 170.0, 1.001, 3.0, 0.45, 8.8]]}

In [22]:
type(input_simple)

dict

In [75]:
import requests

In [118]:
test_1 = requests.post("http://127.0.0.1:4000/predict", json={"input": [[2.0, 0.67, 0.36, 25.7, 0.065, 65.0, 120.0, 1.001, 4.0, 0.65, 6.8],
              [5.0, 0.98, 0.32, 18.9, 0.050, 75.0, 122.0, 0.401, 3.1, 0.21, 1.2]]})

In [109]:
def make_prediction(value: float):
    # Load model
    classifier = joblib.load(MODEL_PATH)
    # Make prediction (the model expects a 2D array that is why we put input in a list of list) and return it
    prediction = classifier.predict(value)
    return prediction

In [110]:
make_prediction([[2.0, 0.67, 0.36, 25.7, 0.065, 65.0, 120.0, 1.001, 4.0, 0.65, 6.8],
              [5.0, 0.98, 0.32, 18.9, 0.050, 75.0, 122.0, 0.401, 3.1, 0.21, 1.2]])

C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator SimpleImputer from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator StandardScaler from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  UserWarning,
C:\Users\javie\anaconda3\lib\site-packages\sklearn\base.py:333: UserWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.0.1 when using version 1.0. This might lead to breaking code or invalid results. Use at

array([5, 6], dtype=int64)

In [111]:
@app.route("/predict", methods=["POST", "GET"])
def predict():
    # Check parameters
    if request.json:
        # Get JSON as dictionnary
        json_input = request.get_json()
        if "input" not in json_input:
            raise MissingKeyError()
        prediction = make_prediction(float(json_input["input"]))
        response = {
            "quality": str(prediction),
        }
        return jsonify(response), 200
    raise MissingJSON()


AssertionError: View function mapping is overwriting an existing endpoint function: predict

In [60]:
@app.route("/")
def index():
    return render_template("index.html")

In [ ]:
if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Restarting with windowsapi reloader


SystemExit: 1